In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import models, datasets, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from torch.utils.data import DataLoader
import numpy as np
from torchinfo import summary
from tqdm import tqdm  
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, roc_auc_score, precision_score, recall_score
from sklearn.metrics import roc_curve


In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available()  else "cpu")
print(device)

cuda:0


In [3]:
data_transform = transforms.Compose(
    [
        transforms.Resize((224,224)),
        transforms.RandomRotation(degrees=30),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406], [0.229,0.224,0.225])
    ]
)

In [4]:
train_dataset = datasets.ImageFolder(root='/kaggle/input/cat-dogs/dogs_vs_cats/train',transform=data_transform)

test_dataset = datasets.ImageFolder(root='/kaggle/input/cat-dogs/dogs_vs_cats/test', transform=data_transform)

In [5]:
print(train_dataset)
print("*"*100)
print(test_dataset)

Dataset ImageFolder
    Number of datapoints: 20000
    Root location: /kaggle/input/cat-dogs/dogs_vs_cats/train
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
               RandomRotation(degrees=[-30.0, 30.0], interpolation=nearest, expand=False, fill=0)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )
****************************************************************************************************
Dataset ImageFolder
    Number of datapoints: 5000
    Root location: /kaggle/input/cat-dogs/dogs_vs_cats/test
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
               RandomRotation(degrees=[-30.0, 30.0], interpolation=nearest, expand=False, fill=0)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.22

In [6]:
train_dataset.classes

['cats', 'dogs']

In [7]:
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=True)

In [8]:
#Iterating over the training dataset and storing the target class for each sample 
classes = [] 
for batch_idx, data in enumerate(train_loader, 0): 
    x, y = data  
    classes.extend(y.tolist()) 
      
#Calculating the unique classes and the respective counts and plotting them 
unique, counts = np.unique(classes, return_counts=True) 

print(unique)
print("")
print(counts)

[0 1]

[10000 10000]


In [9]:
print(len(classes))

20000


In [8]:
test_size = len(test_dataset)
train_size = len(train_dataset)
print(train_size)
print(test_size)

20000
5000


In [9]:
# Get class names
class_names = test_dataset.classes
print(class_names)


['cats', 'dogs']


In [35]:
# Initial model
model_tl = models.alexnet(pretrained=True)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [36]:
model_tl

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [37]:
trainable_params = sum(p.numel() for p in model_tl.parameters() if p.requires_grad)
print(f"Number of trainable parameters: {trainable_params}")


Number of trainable parameters: 61100840


In [38]:
summary(model_tl)

Layer (type:depth-idx)                   Param #
AlexNet                                  --
├─Sequential: 1-1                        --
│    └─Conv2d: 2-1                       23,296
│    └─ReLU: 2-2                         --
│    └─MaxPool2d: 2-3                    --
│    └─Conv2d: 2-4                       307,392
│    └─ReLU: 2-5                         --
│    └─MaxPool2d: 2-6                    --
│    └─Conv2d: 2-7                       663,936
│    └─ReLU: 2-8                         --
│    └─Conv2d: 2-9                       884,992
│    └─ReLU: 2-10                        --
│    └─Conv2d: 2-11                      590,080
│    └─ReLU: 2-12                        --
│    └─MaxPool2d: 2-13                   --
├─AdaptiveAvgPool2d: 1-2                 --
├─Sequential: 1-3                        --
│    └─Dropout: 2-14                     --
│    └─Linear: 2-15                      37,752,832
│    └─ReLU: 2-16                        --
│    └─Dropout: 2-17                   

In [39]:
for param in model_tl.features.parameters():
    param.requires_grad = False

In [40]:
trainable_params = sum(p.numel() for p in model_tl.parameters() if p.requires_grad)
print(f"Number of trainable parameters: {trainable_params}")


Number of trainable parameters: 58631144


In [41]:
# Modify Classifier
num_features = model_tl.classifier[6].in_features
model_tl.classifier[6] = nn.Linear(num_features, 2)

In [42]:
model_tl

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [43]:
trainable_params = sum(p.numel() for p in model_tl.parameters() if p.requires_grad)
print(f"Number of trainable parameters: {trainable_params}")


Number of trainable parameters: 54542338


In [44]:
# Print class to index mapping
print("Class to index mapping:", test_dataset.class_to_idx)  # ADDED

Class to index mapping: {'cats': 0, 'dogs': 1}


In [49]:
# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_tl = model_tl.to(device)
print(device)



cuda


## Evaluation Before doing Transfer learning

In [50]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

model_tl.eval()
correct = 0
total = 0
all_labels = []
all_preds = []

with torch.no_grad():
    for images, labels in tqdm(test_loader, desc="Evaluating"):
        images, labels = images.to(device), labels.to(device)
        outputs = model_tl(images)
        _, predicted = torch.max(outputs, 1)
        
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

        # Store predictions and actual labels
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(predicted.cpu().numpy())

# Compute overall accuracy
accuracy = 100 * correct / total

# Compute additional metrics
precision = precision_score(all_labels, all_preds, average='macro')
recall = recall_score(all_labels, all_preds, average='macro')
f1 = f1_score(all_labels, all_preds, average='macro')
conf_matrix = confusion_matrix(all_labels, all_preds)

# Print results
print(f'Validation Accuracy: {accuracy:.2f}%')
print(f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}')
print(f'Confusion Matrix:\n{conf_matrix}')


Evaluating: 100%|██████████| 625/625 [00:25<00:00, 24.21it/s]

Validation Accuracy: 40.66%
Precision: 0.3983, Recall: 0.4066, F1-score: 0.3942
Confusion Matrix:
[[ 659 1841]
 [1126 1374]]


## Training the Model

In [51]:
# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_tl.parameters(), lr=0.001)

In [52]:
# Training Loop
num_epochs = 5
for epoch in range(num_epochs):
    model_tl.train()
    running_loss = 0.0
    correct = 0
    total = 0
    all_labels = []
    all_preds = []

    for images, labels in tqdm(train_loader, desc="Training"):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model_tl(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

        # Store labels and predictions for metrics
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(predicted.cpu().numpy())

    # Calculate metrics
    accuracy = 100 * correct / total
    precision = precision_score(all_labels, all_preds, average='macro')
    recall = recall_score(all_labels, all_preds, average='macro')
    f1 = f1_score(all_labels, all_preds, average='macro')
    conf_matrix = confusion_matrix(all_labels, all_preds)

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}, Accuracy: {accuracy:.2f}%")
    print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}")
    print(f"Confusion Matrix:\n{conf_matrix}")

print("Training Finished.")

Training: 100%|██████████| 2500/2500 [02:23<00:00, 17.37it/s]


Epoch 1, Loss: 0.4113, Accuracy: 88.48%
Precision: 0.8848, Recall: 0.8848, F1-score: 0.8848
Confusion Matrix:
[[8856 1144]
 [1160 8840]]


Training: 100%|██████████| 2500/2500 [02:23<00:00, 17.39it/s]


Epoch 2, Loss: 0.3004, Accuracy: 90.86%
Precision: 0.9087, Recall: 0.9086, F1-score: 0.9086
Confusion Matrix:
[[9130  870]
 [ 957 9043]]


Training: 100%|██████████| 2500/2500 [02:23<00:00, 17.38it/s]


Epoch 3, Loss: 0.2225, Accuracy: 92.39%
Precision: 0.9239, Recall: 0.9239, F1-score: 0.9238
Confusion Matrix:
[[9313  687]
 [ 836 9164]]


Training: 100%|██████████| 2500/2500 [02:22<00:00, 17.52it/s]


Epoch 4, Loss: 0.2110, Accuracy: 92.47%
Precision: 0.9248, Recall: 0.9247, F1-score: 0.9247
Confusion Matrix:
[[9273  727]
 [ 778 9222]]


Training: 100%|██████████| 2500/2500 [02:22<00:00, 17.49it/s]

Epoch 5, Loss: 0.2268, Accuracy: 92.40%
Precision: 0.9250, Recall: 0.9240, F1-score: 0.9240
Confusion Matrix:
[[9486  514]
 [1006 8994]]
Training Finished.


## Evaluation After Training

In [53]:

model_tl.eval()
correct = 0
total = 0
all_labels = []
all_preds = []

with torch.no_grad():
    for images, labels in tqdm(test_loader, desc="Evaluating"):
        images, labels = images.to(device), labels.to(device)
        outputs = model_tl(images)
        _, predicted = torch.max(outputs, 1)
        
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

        # Store predictions and actual labels
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(predicted.cpu().numpy())

# Compute overall accuracy
accuracy = 100 * correct / total

# Compute additional metrics
precision = precision_score(all_labels, all_preds, average='macro')
recall = recall_score(all_labels, all_preds, average='macro')
f1 = f1_score(all_labels, all_preds, average='macro')
conf_matrix = confusion_matrix(all_labels, all_preds)

# Print results
print(f'Validation Accuracy: {accuracy:.2f}%')
print(f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}')
print(f'Confusion Matrix:\n{conf_matrix}')


Evaluating: 100%|██████████| 625/625 [00:25<00:00, 24.68it/s]

Validation Accuracy: 94.10%
Precision: 0.9425, Recall: 0.9410, F1-score: 0.9410
Confusion Matrix:
[[2425   75]
 [ 220 2280]]


In [65]:
import torch

# Save model_tl's state dictionary
torch.save(model_tl.state_dict(), "Alexnet-finetuned.pth")
print("Model saved as Alexnet-finetuned.pth")


Model saved as Alexnet-finetuned.pth


In [99]:
report_md = """
---
language: en
license: mit
datasets: [Cats-Dogs]
metrics: [accuracy, f1, precision, recall]
---

# 🐱🐶 Transfer Learning on AlexNet for Cats vs. Dogs Classification

This model fine-tunes **AlexNet** using Transfer Learning to classify images into two categories: **Cats** and **Dogs**.

## **📝 Model Details**
- **Pre-trained Model:** AlexNet  
- **Dataset Used:** Cats-Dogs  
- **Batch Size:** 8  
- **Learning Rate:** 0.001  
- **Epochs:** 5  

---

## **📌 Baseline Performance (Before Transfer Learning)**  
**Validation Accuracy:** **40.66%**  
- **Precision:** 0.3983  
- **Recall:** 0.4066  
- **F1-score:** 0.3942  

- Confusion Matrix:
 [[659   1841]
 [1126   1374]]
---

## **✅ Performance After Training**  
**Training Accuracy:** **92.40%**  
- **Precision:** 0.9250  
- **Recall:** 0.9240  
- **F1-score:** 0.9240  

**Confusion Matrix:**
 [[9486   514]
 [1006   8994]]


**Validation Accuracy:** **94.10%**  
- **Precision:** 0.9425  
- **Recall:** 0.9410  
- **F1-score:** 0.9410  

**Confusion Matrix:**
 [[2425   75]
 [ 220   2280]]

---

You can download the model from [Hugging Face](https://huggingface.co/Wolverine001/Alexnet-TransferLearning).
"""


In [74]:
from huggingface_hub import notebook_login

notebook_login(write_permission=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in 'notebook_login': write_permission. Will not be supported from version '1.0'.

Fine-grained tokens added complexity to the permissions, making it irrelevant to check if a token has 'write' access.
  warnings.warn(message, FutureWarning)


In [75]:
from huggingface_hub import HfApi

api = HfApi()

# Create the repository
api.create_repo(
    repo_id="Wolverine001/Alexnet_TransferLearning",  # Change to your username
    repo_type="model",
    exist_ok=True,  # Avoid error if repo already exists
)

print("Repository created successfully!")


Repository created successfully!


In [76]:
# Upload the model file
api.upload_file(
    path_or_fileobj="Alexnet-finetuned.pth",
    path_in_repo="Alexnet-finetuned.pth",
    repo_id="Wolverine001/Alexnet_TransferLearning",
    repo_type="model",
)

# Upload README.md as the model card
api.upload_file(
    path_or_fileobj=report_md.encode(),  
    path_in_repo="README.md",
    repo_id="Wolverine001/Alexnet_TransferLearning",
    repo_type="model",
)

print("Model and README uploaded successfully!")


Alexnet-finetuned.pth:   0%|          | 0.00/228M [00:00<?, ?B/s]

Model and README uploaded successfully!


In [100]:
# Upload README.md as the model card
api.upload_file(
    path_or_fileobj=report_md.encode(),  
    path_in_repo="README.md",
    repo_id="Wolverine001/Alexnet_TransferLearning",
    repo_type="model",
)

print("Model and README uploaded successfully!")

Model and README uploaded successfully!
